In [88]:
import random
import numpy as np
import statsmodels.stats.power as smp
import scipy.stats as stats

# Establish variables
sample_size = 30
significance_level = 0.05 # Alpha
rigging = 2 # 1 = light, 2 = heavy
desired_power = 0.8 # power level target

In [89]:
# A function to generate x amount of dice throws for both a rigged, and fair dice.
def throw_dice(x,rigging=1):
    fair_dice_results = []
    rigged_dice_results = []

    # The lightly rigged dice will produce more 6 values.
    if rigging == 1:
        rigged_dice_numbers = [1, 2, 3, 4, 5, 6, 6]

    # The heavily rigged dice will produce more 4,5,6 values.
    elif rigging == 2:
        rigged_dice_numbers = [1, 2, 3, 4, 4, 5, 5, 6, 6]
        
    else:
        # Handling cases where rigging is not 1 or 2
        raise ValueError("Invalid rigging option. Please choose 1 or 2.")

    for _ in range(x):
        fair_dice_results.append(random.randint(1, 6))
        rigged_dice_results.append(random.choice(rigged_dice_numbers))

    return fair_dice_results, rigged_dice_results

In [90]:
# Example output
fair_dice, rigged_dice = throw_dice(10,1)
print('Example Dice Rolls:')
print(f"Fair Dice Throws: {fair_dice}")
print(f"Rigged Dice Throws: {rigged_dice}")

Example Dice Rolls:
Fair Dice Throws: [5, 2, 2, 6, 5, 4, 5, 4, 4, 4]
Rigged Dice Throws: [3, 4, 1, 4, 4, 5, 6, 5, 6, 1]


In [91]:
def play_games(number_of_games=20,dice_throws=10,rigging=1,show_results=False):
    fair_dice_wins = 0
    rigged_dice_wins = 0
    ties = 0

    for _ in range(number_of_games):
        fair_dice, rigged_dice = throw_dice(dice_throws,rigging)

        if sum(fair_dice) > sum(rigged_dice):
            fair_dice_wins += 1
        elif sum(fair_dice) < sum(rigged_dice):
            rigged_dice_wins += 1
        else:
            ties += 1
            
    if show_results:
        # Displaying the results
        print(f"Games Played: {dice_throws}")
        print(f"Fair Dice Wins: {fair_dice_wins}")
        print(f"Rigged Dice Wins: {rigged_dice_wins}")
        print(f"Ties: {ties}")

    return fair_dice, rigged_dice

In [92]:
def analyze_dice_rolls(sample_size, dice_throws, rigging, significance_level=0.05, desired_power=0.8,show_results=False):

    # Play the games
    fair_dice, rigged_dice = play_games(sample_size, dice_throws, rigging)

    # Calculate the means
    mean_fair = np.mean(fair_dice)
    mean_rigged = np.mean(rigged_dice)

    # Calculate the pooled standard deviation
    std_dev = np.sqrt((np.std(fair_dice, ddof=1) ** 2 + np.std(rigged_dice, ddof=1) ** 2) / 2)

    # Conduct Levene's test for equality of variances
    levene_stat, levene_p = stats.levene(fair_dice, rigged_dice)

    # Calculate Cohen's d
    cohens_d = (mean_fair - mean_rigged) / std_dev

    # Calculate individual standard deviations
    std_dev_fair = np.std(fair_dice, ddof=1)
    std_dev_rigged = np.std(rigged_dice, ddof=1)

    # Calculate Cohen's d for independent standard deviations
    cohens_d_independent = (mean_fair - mean_rigged) / max(std_dev_fair, std_dev_rigged)

    # Perform power analysis
    power_analysis = smp.TTestIndPower()
    power = power_analysis.solve_power(effect_size=cohens_d, nobs1=sample_size, 
                                   alpha=significance_level, ratio=1, 
                                   alternative='two-sided')

    # Calculate sample sizes
    sample_size_pooled = power_analysis.solve_power(effect_size=cohens_d, alpha=significance_level, 
                                                    power=desired_power, ratio=1, 
                                                    alternative='two-sided')
    sample_size_independent = power_analysis.solve_power(effect_size=cohens_d_independent, alpha=significance_level, 
                                                         power=desired_power, ratio=1, 
                                                         alternative='two-sided')
    sample_size_pooled = int(sample_size_pooled)
    sample_size_independent = int(sample_size_independent)

    # Determine which test to use
    test_type = 2 if levene_p < significance_level else 1  # 2 for Welch’s t-test, 1 for standard t-test

    if show_results:
        print('---------------------------------')
        print(f"Mean Fair Throws: {round(mean_fair,1)}")
        print(f"Mean Rigged Throws: {round(mean_rigged,1)}")
        print(f"Pooled Standard Deviation: {round(std_dev,3)}")
        print('---------------------------------')
        print(f"Levene's Test Statistic: {levene_stat}")
        print(f"Levene's Test P-Value: {levene_p}")
        print('---------------------------------')
        print(f"Cohen's d: {round(cohens_d,3)}")
        print(f"Power (%): {round(power*100,1)}")
        print('---------------------------------')
        print(f"Sample Size (Pooled): {round(sample_size_pooled,0)}")
        print(f"Sample Size (Independent): {round(sample_size_independent,0)}")
        print('---------------------------------')

    # Return relevant statistics and test type
    return {
        "sample_size_pooled": sample_size_pooled,
        "sample_size_independent": sample_size_independent,
        "test_type": test_type
    }

In [93]:
def perform_t_test(fair_dice_results, rigged_dice_results, test_type):
    # Choose the appropriate t-test based on the test_type
    if test_type == 1:  # Standard t-test
        t_statistic, p_value = stats.ttest_ind(fair_dice_results, rigged_dice_results, equal_var=True)
        test_name = "Standard t-test"
    elif test_type == 2:  # Welch’s t-test
        t_statistic, p_value = stats.ttest_ind(fair_dice_results, rigged_dice_results, equal_var=False)
        test_name = "Welch’s t-test"
    else:
        raise ValueError("Invalid test type. Choose 1 for Standard t-test or 2 for Welch’s t-test.")

    # Output the t-test results
    print(f"{test_name} Results:")
    print(f"T Statistic: {t_statistic}")
    print(f"P-Value: {p_value}")

In [94]:
# Capture results of initial sample
results = analyze_dice_rolls(sample_size=sample_size, dice_throws=10, rigging=rigging, show_results=True)

# Run dice set off of sample sizes
if results['test_type'] == 1:
    final_sample = results['sample_size_pooled']
else:
    final_sample = results['sample_size_independent']
    
fair_dice_full, rigged_dice_full = play_games(number_of_games=final_sample, dice_throws=10, rigging=rigging, show_results=False)

---------------------------------
Mean Fair Throws: 4.2
Mean Rigged Throws: 4.1
Pooled Standard Deviation: 1.74
---------------------------------
Levene's Test Statistic: 0.6230769230769233
Levene's Test P-Value: 0.44018064295787673
---------------------------------
Cohen's d: 0.057
Power (%): 5.6
---------------------------------
Sample Size (Pooled): 4753
Sample Size (Independent): 5512
---------------------------------


In [95]:
# Run test on final dice sets on the appropriate test
perform_t_test(fair_dice_full, rigged_dice_full, results['test_type'])

Standard t-test Results:
T Statistic: -0.4656903154237996
P-Value: 0.6470224911161582
